In [1]:
%pwd

'c:\\Users\\Khanh\\Desktop\\Medic_Chat\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\Khanh\\Desktop\\Medic_Chat'

In [4]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [6]:
extracted_data = load_pdf_file(data='Data/')

In [7]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [8]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [9]:
from langchain.embeddings import HuggingFaceEmbeddings

In [10]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings


In [11]:
embeddings = download_hugging_face_embeddings()

C:\Users\Khanh\AppData\Local\Temp\ipykernel_2756\1196424635.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\Khanh\anaconda3\envs\Medic_Chat\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
from dotenv import load_dotenv
load_dotenv()

True

In [13]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [14]:
from dotenv import load_dotenv
load_dotenv

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [15]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')

In [16]:
PINECONE_API_KEY 

In [24]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key="pcsk_6N5nSz_HsJKq9mqWoMjAhvNUWbmVyrd3wdTJ5NK4iAfNDyTxFtKbAvhC5G8fBPza6UgUMk")

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [ ]:
import os
os.environ["PINECONE_API_KEY"] = "pcsk_6N5nSz_HsJKq9mqWoMjAhvNUWbmVyrd3wdTJ5NK4iAfNDyTxFtKbAvhC5G8fBPza6UgUMk"
os.environ["GROQ_API_KEY"] = "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"


In [25]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [26]:
docsearch

In [27]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [28]:
retrieved_docs = retriever.invoke("What is Acne?")

In [59]:
retrieved_docs

[Document(id='3bbd7df2-8742-465c-9cc0-3c60ccd68a90', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='74db5f4b-c1d1-4464-ad99-be163d932e15', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM -

In [66]:
from groq import Groq
from langchain.chat_models import ChatOpenAI
import os
llm = ChatOpenAI(
    model="llama3-8b-8192",
    openai_api_base="https://api.groq.com/openai/v1",
    openai_api_key="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",
)


In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [60]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [64]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])

Acromegaly is a disorder caused by the abnormal release of a chemical from the pituitary gland in the brain, leading to increased growth in bone and soft tissue, as well as various bodily disturbances. When this abnormality occurs before bone growth stops, it is called gigantism, but when it occurs after bone growth stops, it is called acromegaly.


In [65]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])

I don't know.


In [ ]:
import os
from dotenv import load_dotenv, find_dotenv


load_dotenv('/Users/Khanh/Desktop/Medic_Chat/.env')
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
print(f"PINECONE_API_KEY: {PINECONE_API_KEY}")

PINECONE_API_KEY: pcsk_6N5nSz_HsJKq9mqWoMjAhvNUWbmVyrd3wdTJ5NK4iAfNDyTxFtKbAvhC5G8fBPza6UgUMk
